In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n_f_list=[4,8,12,14,16,20,24]
rlis=[1.  , 1.02, 1.04, 1.06, 1.08, 1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 ,
       1.22, 1.24, 1.26, 1.28, 1.3 , 1.32, 1.34, 1.36, 1.38, 1.4 , 1.42,
       1.44, 1.46, 1.48, 1.5 , 1.52, 1.54, 1.56, 1.58, 1.6 , 1.62, 1.64,
       1.66, 1.68, 1.7 , 1.72, 1.74, 1.76, 1.78, 1.8 , 1.82, 1.84, 1.86,1.88,1.90,1.92,1.94,1.96,1.98,2.0]

#Load data
fixation_probability=np.zeros([7,len(rlis)])
fixation_time=np.zeros([7,len(rlis)])
extinction_time=np.zeros([7,len(rlis)])
j=0
for n_f in n_f_list:
    i=0
    for r in rlis:
        tit="r="+str(r)+",n_f="+str(n_f)+",fixation time,fixation probability,extincation time.txt"
        x=np.loadtxt(tit)
        fixation_probability[j,i]=x[2]
        fixation_time[j,i]=x[3]
        extinction_time[j,i]=x[4]
        i+=1
    j+=1
Unconditional_time=fixation_probability*fixation_time+(1-fixation_probability)*extinction_time


#Create transition matrix
def transition_matrix(n_f,N,r):
    n_s=N-n_f
    P=np.zeros([N+1,N+1])
    P[0,0]=1
    P[N,N]=1
    for i in range(1,n_f):
        P[i,i+1]=((i*r)/(i*r+N-i))*((n_f-i)/(n_f-1))
        P[i,i-1]=((N-i)/(i*r+N-i))*((n_f-i)/(N-i))*((i)/(n_f-1))
        P[i,i]=1-P[i,i+1]-P[i,i-1]
    i=n1
    P[n_f,n_f-1]=((N-i)/(i*r+N-i))*(1/n_s)*(1/(n_s))
    P[n_f,n_f+1]=((i*r)/(i*r+N-i))*(1/n_f)*(1/(n_f))
    P[n_f,n_f]=1-P[n_f,n_f-1]-P[n_f,n_f+1]
    for i in range(n_f+1,N):
        P[i,i+1]=((i*r)/(i*r+N-i))*((i-n_f)/i)*((N-i)/(n_s-1))
        P[i,i-1]=((N-i)/(i*r+N-i))*((i-n_f)/(n_s-1))
        P[i,i]=1-P[i,i+1]-P[i,i-1]
    return P

#Calculate fixation probability
def Fix_Prob(n_f,N,r,nn):
    p=transition_matrix(n_f,N,r)
    y=0
    for k in range(1,nn):
        x=1
        for j in range(1,k+1):
            x=x*(p[j,j-1]/p[j,j+1])
        y=y+x
    y2=0
    for k in range(1,N):
        x2=1
        for j in range(1,k+1):
            x2=x2*(p[j,j-1]/p[j,j+1])
        y2=y2+x2
    return (1+y)/(1+y2)

#Calculate unconditional fixation time
def unconditional_fixime(r,n_f,N):
    p=transition_matrix(n_f,N,r)
    ft=0
    for k in range(1,N):
        y=0
        for l in range(1,k+1):
            x=1
            for m in range(l+1,k+1):
                x=x*(p[m,m-1]/p[m,m+1])
            y=y+(Fix_Prob(n_f,N,r,1)/p[l,l+1])*x
        ft=ft+y
    return ft

#Calculate conditional fixation time
def conditional_fixime(r,n_f,N):
    p=transition_matrix(n_f,N,r)
    ft=0
    for k in range(1,N):
        y=0
        for l in range(1,k+1):
            x=1
            for m in range(l+1,k+1):
                x=x*(p[m,m-1]/p[m,m+1])
            y=y+(Fix_Prob(n_f,N,r,l)/p[l,l+1])*x
        ft=ft+y
    return ft


Unconditional fixation time

In [ ]:
analitic=np.zeros([7,len(rlis)])
for j in range(7):
    N=28
    n_f=n_f_list[j]
    i=0
    for r in rlis:
        analitic[j,i]=unconditional_fixime(r,n_f,N)
        i+=1
np.savetxt("analitic.txt",analitic)

z1=1
z2=1
z3=0.99
mf=7
lf=1.5
lf2=1.3
xlf=23
ylf=23
lef=18
titf=12
ticfy=15
ticfx=15

color_list=["r","b","m","gold","g","C3","C5"]
marker_list=["o","s","v","P","*","D","x"]
fig=plt.figure(figsize=(30,15))
rect_1=[0,0,.5,.5]
ax1=fig.add_axes(rect_1,facecolor=(z1,z2,z3))
plt.yticks(fontsize=ticfy)
plt.xticks(fontsize=ticfx)
N=28
for j in range(7):
    n_f=n_f_list[j]
    sim1=Unconditional_time[j,:]
    ana1=analitic[j,:]
    marker_interval = 2
    markevery = slice(None, None, marker_interval)
    plt.plot(rlis,sim1,marker=marker_list[j],markevery=markevery,c=color_list[j],markerfacecolor='none',markersize=mf,markeredgewidth=3,linewidth=0)
    plt.plot(rlis,ana1,"-",c=color_list[j],markersize=mf,linewidth=lf)
    plt.plot([-100,-90],[10,30],marker=marker_list[j],c=color_list[j],markerfacecolor='none',label=r"$n_f=$"+str(n_f),markersize=10,linewidth=3)
plt.xlabel(r"$r$",fontsize=xlf)
plt.ylabel("Unconditional fixation time",fontsize=ylf)
plt.grid()
plt.xlim(0.98,2.03)
plt.ylim(170,900)
plt.legend(loc=2,fontsize=lef)
plt.savefig("FIXT,r2.PDF", bbox_inches='tight', dpi=500)

Conditional fixation time

In [ ]:
analitic=np.zeros([7,len(rlis)])
for j in range(7):
    N=28
    n_f=n_f_list[j]
    i=0
    for r in rlis:
        analitic[j,i]=conditional_fixime(r,n_f,N)
        i+=1


z1=1
z2=1
z3=0.99
mf=7
lf=1.5
lf2=1.3
xlf=23
ylf=23
lef=18
titf=12
ticfy=15
ticfx=15


color_list=["r","b","m","gold","g","C3","C5"]
markrt_list=["o","s","v","P","*","D","x"]
fig=plt.figure(figsize=(30,15))
rect_1=[0,0,.5,.5]
ax1=fig.add_axes(rect_1,facecolor=(z1,z2,z3))
plt.yticks(fontsize=ticfy)
plt.xticks(fontsize=ticfx)
N=28
for j in range(7):
    n_f=n_f_list[j]
    sim1=fixation_time[j,:]
    ana1=analitic[j,:]
    marker_interval = 2
    markevery = slice(None, None, marker_interval)
    plt.plot(rlis,sim1,marker=marker_list[j],markevery=markevery,c=color_list[j],markerfacecolor='none',markersize=mf,markeredgewidth=3,linewidth=0)
    plt.plot(rlis,ana1,"-",c=color_list[j],markersize=mf,linewidth=lf)
    plt.plot([-100,-90],[10,30],marker=marker_list[j],c=color_list[j],markerfacecolor='none',label=r"$n_f=$"+str(n_f),markersize=10,linewidth=3)
plt.xlabel(r"$r$",fontsize=xlf)
plt.ylabel("Conditional fixation time",fontsize=ylf)
plt.grid()
plt.xlim(0.98,2.03)
plt.ylim(400,4500)
plt.legend(loc=1,fontsize=lef)
plt.savefig("FIXTC,r2.PDF", bbox_inches='tight', dpi=500)